# Extrapolating Merchant Names from Transaction Descriptions using ML

In my [data engineering project](https://github.com/addenergyx/cashback-data-pipeline), I faced a significant challenge: multiple inconsistent merchant names recorded across transaction data. 
This inconsistency hindered the ability to accurately group transactions, complicating further analysis. 
Given the vast amount of data, manually correcting these discrepancies was impractical. 
To address this, I decided to explore the potential of machine learning to automate the process. 
An unsupervised clustering model was chosen, which is particularly suited for this task because it does not require labeled data, making it feasible to implement across large datasets without prior tagging. 
This approach promises to streamline the preprocessing of transaction data, enhancing the accuracy of the analytical outputs.

In [21]:
import pandas as pd
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

Data has already been preprocessed by the ELT script in the data pipeline [here](https://github.com/addenergyx/cashback-data-pipeline/blob/main/glue_job/elt.py)

In [22]:
data = pd.read_csv('data.csv')

In [23]:
data.head()

,reward_id,transaction_id,description,plu_amount,transaction_date,available,reason,created_at,updated_at,rebate_rate,fiat_amount_rewarded,currency,reference_type,reward_type,amount,plu_price,transaction_amount
0,a299489c-ce6e-40ba-a6d7-026a90e4a510,3e02ef10-9f41-4fd5-9dd1-5b03c455bd96,CURVE SUBSCRIPTION,2.262443,2024-03-23,False,NaN,2024-03-25 06:43:56.024000+00:00,2024-03-25 06:43:56.024000+00:00,0,1000.0,GBP,perk_830246_reward,DAILY_REBATE_DISTRIBUTION,-1799.0,442.000002,17.99
1,3c2fe1f5-c9e1-46e1-941f-00f120abf2d2,fda003db-154a-47e6-8d24-21ab43317814,CRV*PIZZA HUT AIPC HIG,0.304554,2024-03-17,False,NaN,2024-03-19 06:20:47.865000+00:00,2024-03-19 06:20:47.865000+00:00,5,3143.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-3143.0,516.000006,31.43
2,bcc75323-94e8-472b-9feb-d09f89405b7c,030b3eba-0aaf-4b33-861f-a2c93109e204,CRV*SN Gat Airport SST,0.147938,2024-03-17,False,NaN,2024-03-19 06:20:47.630000+00:00,2024-03-19 06:20:47.630000+00:00,5,1435.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-1435.0,485.000014,14.35
3,a01090ad-174c-49ef-9a0e-4b87e5badbdd,32f63642-f1c7-4af7-8753-5deb8ee8dd7d,CRV*M S SIMPLY FOOD SS,0.075356,2024-03-17,False,NaN,2024-03-19 06:20:47.462000+00:00,2024-03-19 06:20:47.462000+00:00,5,740.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-740.0,491.000036,7.40
4,245ca1ed-8af6-4a4c-9a15-62acdf260f29,15996a4b-87c6-47c6-b855-419b477a1c87,CRV*adidas UK Limited,0.579151,2024-03-17,False,NaN,2024-03-19 06:20:47.065000+00:00,2024-03-19 06:20:47.065000+00:00,5,6000.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-6000.0,518.000008,60.00


In [43]:
len(data['description'].unique())

431

## Transaction dataset review and further preprocessing

Transactions made with a Curve card will appear as CRV* – “Merchant Name” (for example: CRV* – Starbucks) on bank statements.

In [44]:
data['description'] = data['description'].astype(str)
data['description'] = data['description'].apply(lambda x: x.replace('CRV*', ''))
data['description'] = data['description'].apply(lambda x: x.replace('ZILCH*', ''))

In [45]:
df2 = data.loc[data['description'] != None]

description_trends = {}

most_common = {}

for ind in df2.index:
    description = df2['description'][ind]
    if description in description_trends:
        description_trends[description] += 1 
    else:
        description_trends[description] = 1
    most_common = Counter(description_trends).most_common(50)  

In [46]:
most_common

[('Klarna, London, GB CD 4309', 512),
 ('Klarna', 338),
 ('ZILCH., London, GB CD 4309', 121),
 ('ARCADE CAFE, London, GB CD 4309', 13),
 ('UBR* PENDING.UBER.COM, London, GB CD 4309', 12),
 ('CURVE SUBSCRIPTION, 00012, LONDON, GB CD 4309', 10),
 ('Cinnabar, London, GB CD 4309', 9),
 ('CURVE SUBSCRIPTION', 8),
 ('APPLE.COM BILL, London, GB CD 4309', 8),
 ('AWS EMEA, London, GB CD 4309', 8),
 ('WAIPO EAT STOCK, London, GB CD 4309', 8),
 ('BUTLINS CENTRE STAGE, London, GB CD 4309', 8),
 ('adidas UK Limited', 6),
 ('AIRTIME REWARDS LTD, London, GB CD 4309', 6),
 ('AIRTIME REWARDS LTD, 00025, MANCHESTER, GB CD 4309', 6),
 ('DRAPER ARMS, London, GB CD 4309', 6),
 ('WAYBETTER  INC., London, GB CD 4309', 6),
 ('ASDA CS LTD MONKSWOOD', 5),
 ('Audible UK', 5),
 ('ASDA CS LTD\\MONKSWOOD', 5),
 ('ASDA CS LTD, London, GB CD 4309', 5),
 ('Trainline, London, GB CD 4309', 5),
 ('CINEWORLD, London, GB CD 4309', 5),
 ('SLUG   LETTUCE   POPW, London, GB CD 4309', 5),
 ('TESCO PFS 3891, London, GB CD 4309'

In [47]:
# Possible merchant names from descriptions
merchant_keywords = [
    'TESCO', 'ASDA', 'COSTCO', 'SPOTIFY', 'AMAZON', 'APPLE', 'NIKE', 'ADIDAS',
    'KFC', 'MCDONALDS', 'SAINSBURYS', 'PIZZA HUT', 'UBER', 'PAYPAL', 'EBAY',
    'ARGOS', 'BOOTS', 'HERMES', 'JADE GARDEN', 'CINNABAR', 'ZILCH', 'KLARNA', 'BUTLINS', 'CINEWORLD'
    'CURVE', 'AIRTIME REWARDS'
]

In [48]:
# Function to extract merchant name from description
def extract_merchant(description):
    description = description.upper()
    for keyword in merchant_keywords:
        if keyword in description:
            return keyword
    return description  # Fallback category for unmatched descriptions

In [49]:
data['merchant_name'] = data['description'].apply(extract_merchant)

In [50]:
data.head()

,reward_id,transaction_id,description,plu_amount,transaction_date,available,reason,created_at,updated_at,rebate_rate,fiat_amount_rewarded,currency,reference_type,reward_type,amount,plu_price,transaction_amount,merchant_name
0,a299489c-ce6e-40ba-a6d7-026a90e4a510,3e02ef10-9f41-4fd5-9dd1-5b03c455bd96,CURVE SUBSCRIPTION,2.262443,2024-03-23,False,NaN,2024-03-25 06:43:56.024000+00:00,2024-03-25 06:43:56.024000+00:00,0,1000.0,GBP,perk_830246_reward,DAILY_REBATE_DISTRIBUTION,-1799.0,442.000002,17.99,CURVE SUBSCRIPTION
1,3c2fe1f5-c9e1-46e1-941f-00f120abf2d2,fda003db-154a-47e6-8d24-21ab43317814,PIZZA HUT AIPC HIG,0.304554,2024-03-17,False,NaN,2024-03-19 06:20:47.865000+00:00,2024-03-19 06:20:47.865000+00:00,5,3143.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-3143.0,516.000006,31.43,PIZZA HUT
2,bcc75323-94e8-472b-9feb-d09f89405b7c,030b3eba-0aaf-4b33-861f-a2c93109e204,SN Gat Airport SST,0.147938,2024-03-17,False,NaN,2024-03-19 06:20:47.630000+00:00,2024-03-19 06:20:47.630000+00:00,5,1435.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-1435.0,485.000014,14.35,SN GAT AIRPORT SST
3,a01090ad-174c-49ef-9a0e-4b87e5badbdd,32f63642-f1c7-4af7-8753-5deb8ee8dd7d,M S SIMPLY FOOD SS,0.075356,2024-03-17,False,NaN,2024-03-19 06:20:47.462000+00:00,2024-03-19 06:20:47.462000+00:00,5,740.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-740.0,491.000036,7.40,M S SIMPLY FOOD SS
4,245ca1ed-8af6-4a4c-9a15-62acdf260f29,15996a4b-87c6-47c6-b855-419b477a1c87,adidas UK Limited,0.579151,2024-03-17,False,NaN,2024-03-19 06:20:47.065000+00:00,2024-03-19 06:20:47.065000+00:00,5,6000.0,GBP,fiat_transactions,DAILY_REBATE_DISTRIBUTION,-6000.0,518.000008,60.00,ADIDAS


In [51]:
data.isnull().sum()

reward_id                0
transaction_id           0
description              0
plu_amount               0
transaction_date         0
available                0
reason                  48
created_at               0
updated_at               0
rebate_rate              0
fiat_amount_rewarded     0
currency                 0
reference_type           0
reward_type              0
amount                   0
plu_price                0
transaction_amount       0
merchant_name            0
dtype: int64

In [52]:
data['merchant_name'].unique()

array(['CURVE SUBSCRIPTION', 'PIZZA HUT', 'SN GAT AIRPORT SST',
       'M S SIMPLY FOOD SS', 'ADIDAS', 'STOCKX MARKETPLACE NL', 'PAYPAL',
       'EXPERIAN UK NOTTIN', 'BOOHOO.COM UK LTD', 'RENDERCOM 14158304',
       'NANDOSCOUK 4420839', 'CKMNORD VPNCOM GBR', 'BRITISH A 35314369',
       'NIKE', 'STEVENAGE BOROUGH', 'COLLECTION POTUNIT',
       'YUM BAOFLAT 16, WE', 'MCDONALDS', 'AMAZON', 'AMZNMKTPLACE1 PRIN',
       'EBAY', 'SPOTIFY', 'OFFICE', 'HOME BARGAINS STEV',
       'RIU HOTELS AND RES', 'SIZE?', 'APPLE', 'COSTCO',
       'DART CHARGE AUTO TOP', 'HEROKU* JAN 921682', 'EXPERIAN UKEXPERIA',
       'TESCO', 'TRAINLINETRAINLINE', 'AWS EMEA38 AVENUE',
       'TOOTH CLUB STEVENA', 'ALDI 40 776 FAIRLA', 'BARBICAN EVENTS LT',
       'WHITE HART - HERTF', 'CLOUD 396VQR', 'TRAINLINE.COM LOND',
       'JUMBO UK LTD LONDO', 'WANIS INTERNATIONA', 'NSPPD LAGOS NGA',
       'GOOGLE*CLOUD TKJLW', 'STEVENAGE STEVENAG', 'NSPPD LAGOS LAGOS',
       'JADE GARDEN', 'CINNABAR', 'SUMUP *DIRTIES 61',

In [53]:
len(data['merchant_name'].unique())

314

In [54]:
# Encode merchant names
label_encoder = LabelEncoder()
merchant_labels = label_encoder.fit_transform(data['merchant_name'])

In [55]:
# K-Means clustering, one cluster per unique merchant
kmeans = KMeans(n_clusters=len(label_encoder.classes_), random_state=42)
kmeans.fit(merchant_labels.reshape(-1, 1))
data['cluster'] = kmeans.labels_

In [56]:
# Analyze clusters
transaction_counts = data.groupby(['cluster', 'merchant_name']).size().reset_index(name='Number of Transactions')
sample_descriptions = data.groupby(['cluster', 'merchant_name'])['description'].apply(
    lambda x: x.sample(min(len(x), 5))).reset_index().rename(columns={'level_2': 'original_index'})
detailed_cluster_info = pd.merge(transaction_counts, sample_descriptions, on=['cluster', 'merchant_name'])

In [57]:
pd.set_option('display.max_columns', None)
detailed_cluster_info

,cluster,merchant_name,Number of Transactions,original_index,description
0,0,"HM SE0187, LONDON, GB CD 4309",1,1126,"HM SE0187, London, GB CD 4309"
1,1,TSB RETURNS LTD,1,144,TSB Returns Ltd
2,2,BUTLINS,28,1572,"BUTLINS CENTRE STAGE, London, GB CD 4309"
3,2,BUTLINS,28,1586,"BUTLINS BAR ROSSO, London, GB CD 4309"
4,2,BUTLINS,28,1580,"BUTLINS BURGER KING, London, GB CD 4309"
...,...,...,...,...,...
517,309,"VASAMUSEETS ENTRE, LONDON, GB CD 4309",1,1095,"VASAMUSEETS ENTRE, London, GB CD 4309"
518,310,"AH TO GO 5817, LONDON, GB CD 4309",1,871,"Ah To Go 5817, London, GB CD 4309"
519,311,"YUM BAOFLAT 16, WE",1,19,"Yum baoFlat 16, We"
520,312,"WEMBLEY PARK, LONDON, GB CD 4309",1,713,"WEMBLEY PARK, London, GB CD 4309"


In [58]:
detailed_cluster_info.to_csv('clusters.csv')